Cetta Maulana Andhika
1103213119
TK-45-04
dataset : credit_score.csv

In [53]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.preprocessing import StandardScaler,LabelEncoder
from sklearn.metrics import accuracy_score, classification_report,mean_absolute_error,mean_absolute_percentage_error,mean_squared_error
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
import xgboost as xgb



In [54]:
data = pd.read_csv("datatset\\train.csv")
data = data.drop(columns=['ID',"Customer_ID","SSN","Credit_History_Age",'Name',"Type_of_Loan"])
data = data.dropna(axis=0)

data = data.applymap(lambda x: x.replace('_', '') if isinstance(x, str) else x)
data.describe()

C:\Users\cetta\AppData\Local\Temp\ipykernel_11928\1453913884.py:1: DtypeWarning: Columns (26) have mixed types. Specify dtype option on import or set low_memory=False.
  data = pd.read_csv("datatset\\train.csv")


,Monthly_Inhand_Salary,Num_Bank_Accounts,Num_Credit_Card,Interest_Rate,Delay_from_due_date,Num_Credit_Inquiries,Credit_Utilization_Ratio,Total_EMI_per_month
count,73156.000000,73156.000000,73156.000000,73156.000000,73156.000000,73156.000000,73156.000000,73156.000000
mean,4152.950829,16.798472,22.302409,73.996774,21.077041,27.380037,32.323526,1416.766806
std,3166.097825,115.919146,127.945339,473.014329,14.834577,191.617437,5.107755,8345.826687
min,303.645417,-1.000000,0.000000,1.000000,-5.000000,0.000000,20.881250,0.000000
25%,1619.936667,3.000000,4.000000,8.000000,10.000000,3.000000,28.095946,30.014601
50%,3055.784583,6.000000,6.000000,14.000000,18.000000,6.000000,32.359434,68.240489
75%,5903.517500,7.000000,7.000000,20.000000,28.000000,9.000000,36.527525,155.501023
max,15204.633333,1798.000000,1499.000000,5797.000000,67.000000,2597.000000,50.000000,82331.000000


In [55]:
categorical = ['Month',"Occupation","Credit_Mix","Payment_of_Min_Amount","Payment_Behaviour","Credit_Score"]
label_encoder = LabelEncoder()
for x in categorical:
    data[x] = label_encoder.fit_transform(data[x])


In [56]:
# Filter out negative values in numeric columns
data = data.apply(pd.to_numeric,errors='coerce')
data = data.applymap(lambda x: x if x >= 0 else None)
data.dropna(axis=0,inplace=True)

In [57]:
# Calculate quartiles for each feature
quartiles = data.quantile([0.25, 0.75])

# Calculate IQR (Interquartile Range) for each feature
IQR = quartiles.loc[0.75] - quartiles.loc[0.25]

# Define lower and upper bounds for each feature
lower_bound = (quartiles.loc[0.25] - 1.5 * IQR).to_dict()
upper_bound = (quartiles.loc[0.75] + 1.5 * IQR).to_dict()

# Filter out data points outside the bounds for each feature
filtered_data = data.copy()
for feature in data.columns:
    lower_bound_value = lower_bound[feature]
    upper_bound_value = upper_bound[feature]
    filtered_data = filtered_data[(filtered_data[feature] >= lower_bound_value) & (filtered_data[feature] <= upper_bound_value)]
data = filtered_data


In [58]:
# Separate features (X) and target variable (y)
X = data.drop('Credit_Score', axis=1)  # Replace 'target_variable_name' with the name of your target variable
y = data['Credit_Score']

# Split the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=42)

# Feature scaling
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Initialize KNN classifier
knn_classifier = KNeighborsClassifier(n_neighbors=10)  # You can adjust the number of neighbors

# Train the KNN classifier
knn_classifier.fit(X_train_scaled, y_train)

# Predict the labels for test set
y_pred = knn_classifier.predict(X_test_scaled)

# Evaluate the model
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)

# Classification report
print("\nClassification Report:")
print(classification_report(y_test, y_pred))
mean_error = mean_absolute_error(y_test, y_pred)
print("Mean Absolute Error:", mean_error)
mape = mean_absolute_percentage_error(y_test, y_pred)
print("Mean Absolute Percentage Error (MAPE):", mape)
mse = mean_squared_error(y_test, y_pred)
print("Mean Squared Error (MSE):", mse)

Accuracy: 0.7089895808093046

Classification Report:
              precision    recall  f1-score   support

           0       0.56      0.57      0.56       674
           1       0.71      0.67      0.69      1185
           2       0.75      0.77      0.76      2268

    accuracy                           0.71      4127
   macro avg       0.67      0.67      0.67      4127
weighted avg       0.71      0.71      0.71      4127

Mean Absolute Error: 0.4099830385267749
Mean Absolute Percentage Error (MAPE): 613283981241148.2
Mean Squared Error (MSE): 0.6479282771989339


In [59]:
# Split the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Initialize Random Forest classifier
rf_classifier = RandomForestClassifier(n_estimators=100, random_state=42)  # You can adjust the number of estimators

# Train the Random Forest classifier
rf_classifier.fit(X_train, y_train)

# Predict the labels for test set
y_pred = rf_classifier.predict(X_test)

# Evaluate the model
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)

# Classification report
print("\nClassification Report:")
print(classification_report(y_test, y_pred))
mean_error = mean_absolute_error(y_test, y_pred)
print("Mean Absolute Error:", mean_error)
mape = mean_absolute_percentage_error(y_test, y_pred)
print("Mean Absolute Percentage Error (MAPE):", mape)
mse = mean_squared_error(y_test, y_pred)
print("Mean Squared Error (MSE):", mse)

Accuracy: 0.7872546644051369

Classification Report:
              precision    recall  f1-score   support

           0       0.73      0.69      0.71      1323
           1       0.77      0.79      0.78      2353
           2       0.81      0.81      0.81      4578

    accuracy                           0.79      8254
   macro avg       0.77      0.76      0.77      8254
weighted avg       0.79      0.79      0.79      8254

Mean Absolute Error: 0.2984007753816331
Mean Absolute Percentage Error (MAPE): 437592306899823.0
Mean Squared Error (MSE): 0.46971165495517325


In [60]:
# Initialize XGBoost classifier
xgb_classifier = xgb.XGBClassifier(objective='binary:logistic', random_state=42)  

# Train the XGBoost classifier
xgb_classifier.fit(X_train, y_train)

# Predict the labels for test set
y_pred = xgb_classifier.predict(X_test)

# Evaluate the model
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)

# Classification report
print("\nClassification Report:")
print(classification_report(y_test, y_pred))
mean_error = mean_absolute_error(y_test, y_pred)
print("Mean Absolute Error:", mean_error)
mape = mean_absolute_percentage_error(y_test, y_pred)
print("Mean Absolute Percentage Error (MAPE):", mape)
mse = mean_squared_error(y_test, y_pred)
print("Mean Squared Error (MSE):", mse)

Accuracy: 0.7740489459655925

Classification Report:
              precision    recall  f1-score   support

           0       0.69      0.67      0.68      1323
           1       0.76      0.76      0.76      2353
           2       0.80      0.81      0.81      4578

    accuracy                           0.77      8254
   macro avg       0.75      0.75      0.75      8254
weighted avg       0.77      0.77      0.77      8254

Mean Absolute Error: 0.31693724254906713
Mean Absolute Percentage Error (MAPE): 472512391240955.9
Mean Squared Error (MSE): 0.49890961957838625
